In [1]:
from data_processor import DataProcessor
from nuplan.common.actor_state.state_representation import Point2D
from nuplan.planning.scenario_builder.nuplan_db.nuplan_scenario_utils import ScenarioMapping
from nuplan.planning.scenario_builder.nuplan_db.nuplan_scenario_builder import NuPlanScenarioBuilder
from nuplan.planning.scenario_builder.scenario_filter import ScenarioFilter
from nuplan.planning.utils.multithreading.worker_parallel import SingleMachineParallelExecutor
from diffusion_planner.data_process.utils import (
get_scenario_map,
get_filter_parameters,
sampled_tracked_objects_to_tensor
)
import numpy as np


In [2]:
db_path = "/cailiu2/Diffusion-Planner/data/2021.10.21.14.43.30_veh-28_01244_01519.db" # single db file
db_path = "/share/data_cold/open_data/nuplan/data/cache/mini"  # multi db files
data_root = "/cailiu2/Diffusion-Planner/data/"
map_path = "/cailiu2/Diffusion-Planner/data/maps"
map_version = "nuplan-maps-v1.0"

save_processed_path = "/cailiu2/Diffusion-Planner/data/processed"
total_scenarios = 10
scenario_mapping = ScenarioMapping(scenario_map=get_scenario_map(), subsample_ratio_override=0.5)
builder = NuPlanScenarioBuilder(db_path, map_path, None, None, map_version, scenario_mapping = scenario_mapping)
worker = SingleMachineParallelExecutor(use_process_pool=True)
scenario_filter = ScenarioFilter(*get_filter_parameters(num_scenarios_per_type=30000,
                                                          limit_total_scenarios=total_scenarios))
scenarios = builder.get_scenarios(scenario_filter, worker)
del worker, builder, scenario_filter
processor = DataProcessor(scenarios)

In [5]:

ego_agent_past = None # inference no need ego_agent_past
processor.scenario = scenarios[0]


In [ ]:
"""
ego
"""
ego_state = processor.scenario.initial_ego_state
ego_coords = Point2D(ego_state.rear_axle.x, ego_state.rear_axle.y)
anchor_ego_state = np.array([ego_state.rear_axle.x, ego_state.rear_axle.y, ego_state.rear_axle.heading], dtype=np.float64)
print(ego_coords)
print(anchor_ego_state)


"""
neighbor #scenario.get_past_tracked_objects(iteration=10, time_horizon=2.0)
"""
# neighbor_agent_past, neighbor_agents_types = processor.get_neighbor_agents()
neighbors = processor.scenario.initial_tracked_objects
neighbors.tracked_objects
print(len(neighbors.tracked_objects))
print(type(neighbors.tracked_objects))

neighbor_list =[]

# for neighbor in neighbors:
#   neighbor_list.append(neighbor)
#   print(neighbor)
tracked_objects = sampled_tracked_objects_to_tensor(neighbors.tracked_objects)
print(type(tracked_objects))


Point2D(x=330957.49299200956, y=4691143.579406448)
[3.30957493e+05 4.69114358e+06 5.43869085e-01]
58
<class 'nuplan.common.actor_state.tracked_objects.TrackedObjects'>
<class 'tuple'>
